# Quadrat-based statistics in `pointpats`

This notebook provides an introduction to **quadrat-based methods** for planar
point patterns, focusing on:

- The intuition behind quadrat counts for testing **complete spatial randomness** (CSR)
- The `QStatistic` class in `pointpats.quadrat_statistics`
- Analytical and empirical (simulation-based) chi-squared tests under CSR
- Rectangular and hexagonal tessellations for the study region

We will:

1. Review the quadrat-based approach to testing a point pattern for CSR.
2. Work through a juvenile point-pattern example using `QStatistic`, comparing
   analytical and simulation-based inference.
3. Explore how changing the tessellation (rectangles vs. hexagons) affects the
   quadrat test and its interpretation.

This notebook is designed to live in the `docs/user-guide/` folder and be
executed automatically as part of the `pointpats` documentation build.


## 1. Quadrat statistics and CSR

In the previous notebooks, we introduced the concept of Complete Spatial Randomness (CSR) process which serves as the benchmark process. Utilizing CSR properties, we can discriminate those that are not from a CSR process. Quadrat statistic is one such method. Since a CSR process has two major characteristics:
1. Uniform: each location has equal probability of getting a point (where an event happens).
2. Independent: location of event points are independent.

We can imagine that for any point pattern, if the underlying process is a CSR process, the expected point counts inside any cell of area $|A|$ should be $\lambda |A|$ ($\lambda$ is the intensity which is uniform across the study area for a CSR). Thus, if we impose a $m \times k$ rectangular tessellation over the study area (window), we can easily calculate the expected number of points inside each cell under the null of CSR. By comparing the observed point counts against the expected counts and calculate a $\chi^2$ test statistic, we can decide whether to reject the null based on the position of the $\chi^2$ test statistic in the sampling distribution. 

$$\chi^2 = \sum^m_{i=1} \sum^k_{j=1} \frac{[x_{i,j}-E(x_{i,j})]^2}{\lambda |A_{i,j}|}$$

There are two ways to construct the sampling distribution and acquire a p-value:
1. Analytical sampling distribution: a $\chi^2$ distribution of $m \times k -1$ degree of freedom. We can refer to the $\chi^2$ distribution table to acquire the p-value. If it is smaller than $0.05$, we will reject the null at the $95\%$ confidence level.
2. Empirical sampling distribution: a distribution constructed from a large number of $\chi^2$ test statistics for  simulations under the null of CSR. If the $\chi^2$ test statistic for the observed point pattern is among the largest $5%$ test statistics, we would say that it is very unlikely that it is the outcome of a CSR process at the $95\%$ confidence level. Then, the null is rejected. A pseudo p-value can be calculated based on which we can use the same rule as p-value to make the decision:
$$p(\chi^2) = \frac{1+\sum^{nsim}_{i=1}\phi_i}{nsim+1}$$
where 
$$ 
\phi_i =
 \begin{cases}
    1       & \quad \text{if } \psi_i^2 \geq \chi^2 \\
    0       & \quad \text{otherwise } \\
  \end{cases}
$$

$nsim$ is the number of simulations, $\psi_i^2$ is the $\chi^2$ test statistic calculated for each simulated point pattern, $\chi^2$ is the $\chi^2$ test statistic calculated for the observed point pattern, $\phi_i$ is an indicator variable.

We are going to introduce how to use the **quadrat_statistics.py** module to perform quadrat based method using either of the above two approaches to constructing the sampling distribution and acquire a p-value.


## 2. Juvenile example

In [ ]:
import libpysal as ps
import numpy as np
from pointpats import PointPattern, as_window
from pointpats import PoissonPointProcess as csr
%matplotlib inline
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

Import the quadrat_statistics module to conduct quadrat-based method. 

Among the three major classes in the module, **RectangleM, HexagonM, QStatistic**, the first two are aimed at imposing a tessellation (rectangular or hexagonal shape) over the minimum bounding rectangle of the point pattern and calculate the number of points falling in each cell; **QStatistic** is the main class with which we can calculate a p-value, as well as a pseudo p-value to help us make the decision of rejecting the null or not.

In [ ]:
import pointpats.quadrat_statistics as qs

In [ ]:
dir(qs)

Open the point shapefile "juvenile.shp".

In [ ]:
juv = ps.io.open(ps.examples.get_path("juvenile.shp"))

In [ ]:
len(juv) # 168 point events in total

In [ ]:
juv_points = np.array([event for event in juv]) # get x,y coordinates for all the points
juv_points

Construct a point pattern from numpy array **juv_points**.

In [ ]:
pp_juv = PointPattern(juv_points)
pp_juv

In [ ]:
pp_juv.summary()

In [ ]:
pp_juv.plot(window= True, title= "Point pattern")

### 2.1 Rectangle quadrats & analytical sampling distribution

We can impose rectangle tessellation over mbb of the point pattern by specifying **shape** as "rectangle". We can also specify the number of rectangles in each row and column. For the current analysis, we use the $3 \times 3$ rectangle grids.

In [ ]:
q_r = qs.QStatistic(pp_juv,shape= "rectangle",nx = 3, ny = 3)

Use the plot method to plot the quadrats as well as the number of points falling in each quadrat.

In [ ]:
q_r.plot()

In [ ]:
q_r.mr.rectangle_width * q_r.mr.rectangle_height #calculate the area of each grid cell 

In [ ]:
q_r.chi2 #chi-squared test statistic for the observed point pattern

In [ ]:
q_r.df #degree of freedom

In [ ]:
q_r.chi2_pvalue # analytical pvalue

Since the p-value based on the analytical $\chi^2$ distribution (degree of freedom = 8) is 0.0000589, much smaller than 0.05. We might determine that the underlying process is not CSR. We can also turn to empirical sampling distribution to ascertain our decision.

### 2.2 Rectangle quadrats & empirical sampling distribution

To construct a empirical sampling distribution, we need to simulate CSR within the window of the observed point pattern a lot of times. Here, we generate 999 point patterns under the null of CSR.

In [ ]:
csr_process = csr(pp_juv.window, pp_juv.n, 999, asPP=True)

We specify parameter **realizations** as the point process instance which contains 999 CSR realizations.

In [ ]:
q_r_e = qs.QStatistic(pp_juv,shape= "rectangle",nx = 3, ny = 3, realizations = csr_process)

In [ ]:
q_r_e.chi2_r_pvalue

The pseudo p-value is 0.002, which is smaller than 0.05. Thus, we reject the null at the $95\%$ confidence level.

### 2.3 Hexagon quadrats & analytical sampling distribution

We can also impose hexagon tessellation over mbb of the point pattern by specifying **shape** as "hexagon". We can also specify the length of the hexagon edge. For the current analysis, we specify it as 15.

In [ ]:
q_h = qs.QStatistic(pp_juv,shape= "hexagon",lh = 15)

In [ ]:
q_h.plot()

In [ ]:
q_h.chi2 #chi-squared test statistic for the observed point pattern

In [ ]:
q_h.df #degree of freedom

In [ ]:
q_h.chi2_pvalue  # analytical pvalue

Similar to the inference of rectangle tessellation, since the analytical p-value is much smaller than 0.05, we reject the null of CSR. The point pattern is not random.

### 2.4 Hexagon quadrats & empirical sampling distribution

In [ ]:
q_h_e = qs.QStatistic(pp_juv,shape= "hexagon",lh = 15, realizations = csr_process)

In [ ]:
q_h_e.chi2_r_pvalue

Because 0.001 is smaller than 0.05, we reject the null.

## 3. Recap

In this notebook, we:

- Introduced the use of **quadrat counts** to test a point pattern for
  complete spatial randomness (CSR).
- Used the `QStatistic` class in `pointpats` to compute the quadrat test
  statistic and its **analytical chi-squared p-value** under CSR.
- Compared analytical inference with **simulation-based (Monte Carlo)**
  inference using repeated CSR realizations.
- Explored how the **choice of tessellation** (rectangles vs. hexagons) can
  affect the test outcome and its interpretation.

Together with the distance-based statistics notebook (F, G, K, L), this
provides a complementary, intensity-based view of spatial point pattern
structure in `pointpats`.
